In [1]:
import sys
sys.path.append('../convertToScreen/')
sys.path.append('../../30_data_tools/')

In [2]:
import pandas as pd
import sqlite3
from pathlib import Path

In [3]:
from update_generic_images_db import load_data_to_process, write_generic_images
from helper import load_dotenv

In [4]:
dotenv = load_dotenv()

In [5]:
con = sqlite3.connect( dotenv['DB_PATH'] )

In [6]:
df = load_data_to_process( dotenv )

In [32]:
def write_masks( data, con ):
    already_processed_masks = pd.read_sql(
        '''
            SELECT
                pdf_filename,
                job,
                "type",
                variant_name,
                method,
                idx,
                bbox as bbox_string,
                1 AS is_already_processed
            FROM mask
        ''',
        con
    )
    already_processed_masks.idx = already_processed_masks.idx.astype('str')

    merged = pd.merge(
        data,
        already_processed_masks,
        how="left",
        on=[
            'pdf_filename',
            'job',
            'type',
            'variant_name',
            'method',
            'idx',
            'bbox_string'
        ]
    )
    merged.is_already_processed = merged.is_already_processed == 1

    sql_lines = [
        f"('{ row.pdf_filename }','{ row.job }','{ row.type }','{ row.variant_name }','{ row.method }',{ row.idx },'{ row['id'] }','{ row.bbox_string }',{ row.ssim },{ row.overlay_intensity_C },{ row.overlay_intensity_M },{ row.overlay_intensity_Y },{ row.overlay_intensity_K })"
        for _,row in merged.loc[merged.is_already_processed == False].iterrows()
    ]

    errors = 0
    if len(sql_lines) > 0:
        SQL = f'''
            INSERT INTO mask ('pdf_filename','job','type','variant_name','method','idx','mask_id','bbox','ssim','overlay_intensity_C','overlay_intensity_M','overlay_intensity_Y','overlay_intensity_K')
            VALUES { ",".join(sql_lines) }
        '''
        
        c = con.cursor()
        try:
            c.execute(SQL)
        except sqlite3.IntegrityError:
            for l in sql_lines:
                try:
                    SQL = f'''
                        INSERT INTO mask ('pdf_filename','job','type','variant_name','method','idx','mask_id','bbox','ssim','overlay_intensity_C','overlay_intensity_M','overlay_intensity_Y','overlay_intensity_K')
                        VALUES { l }
                    '''
                    c.execute(SQL)
                except sqlite3.IntegrityError:
                    errors += 1
        
        
        c.close()
        con.commit()

        print(f"successfully written { (len(sql_lines) - errors) }/{ len(sql_lines) } masks")

In [33]:
write_masks( df, con )